# El Yapımı Standardizer

🧑🏻‍🏫 Bu challenge’da *kendi* StandardScaler’ımızı oluşturacağız. Neden mi? Güzel soru!

🎯 Bu egzersizin amaçları şunlardır:
- `stateless transformers` ile `stateful transformers` arasındaki farkı anlamak
- `FeatureUnion` ile çalışmak

## (1) 📚 Stateless Transformer vs. Stateful Transformer

🔢 Aşağıdaki training set’i ve aşağıdaki test set’i göz önünde bulundurun...

In [87]:
import numpy as np
import pandas as pd

X_train = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 4, 1: 5, 2: 6},
    'C': {0: 7, 1: 8, 2: 9}
})

print("This is the training dataset:")
display(X_train)

X_test = pd.DataFrame({
    'A': {0: 1, 1: 2, 2: 3},
    'B': {0: 2, 1: 3, 2: 4},
    'C': {0: 3, 1: 4, 2: 10}
})

print("This is the test dataset:")
display(X_test)

This is the training dataset:


,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


This is the test dataset:


,A,B,C
0,1,2,3
1,2,3,4
2,3,4,10


🛠 ...ve aşağıdaki FeatureUnion’ı düşünün; bu yapı:
- feature’ları ölçekler
- diğer (ölçeklenmemiş) feature’ların ortalaması olan yeni bir feature oluşturur

In [88]:
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline, make_union, Pipeline, FeatureUnion


standard_scaler = StandardScaler()
feature_averager = FunctionTransformer(lambda df: pd.DataFrame(1/3 * (df["A"] + df["B"] + df["C"])))

pipeline = make_union(standard_scaler, feature_averager)
pipeline

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x7e66cfa46660>))])

Hadi:
- pipeline’ı training set üzerinde fit edelim
- hem training set’i hem de test set’i transform edelim

In [89]:
pipeline.fit(X_train)

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('functiontransformer',
                                FunctionTransformer(func=<function <lambda> at 0x7e66cfa46660>))])

In [90]:
X_train_transformed = pd.DataFrame(pipeline.transform(X_train))
X_train_transformed

,0,1,2,3
0,-1.224745,-1.224745,-1.224745,4.0
1,0.000000,0.000000,0.000000,5.0
2,1.224745,1.224745,1.224745,6.0


In [91]:
X_test_transformed = pd.DataFrame(pipeline.transform(X_test))
X_test_transformed

,0,1,2,3
0,-1.224745,-3.674235,-6.123724,2.000000
1,0.000000,-2.449490,-4.898979,3.000000
2,1.224745,-1.224745,2.449490,5.666667


👨🏻‍🏫 `StandardScaler` ile `FunctionTransformer`’ın temelde ne kadar farklı olduğuna dikkat edin ❗️

Pipeline’ı fit edip hem training set’i hem de test set’i transform ettiğimizde:

* **`FunctionTransformer (feature_averager)`**:
    * *.fit()* sırasında hiçbir şey “öğrenmedi”
    * yalnızca **stateless bir dönüşüm** uyguladı: $ \large (X_1, X_2, X_3) \rightarrow \frac{(X_1 + X_2 + X_3)}{3}$


* **`StandardScaler`**:
    * *.fit()* sırasında $\mu_{\color{blue}{train}}$ ve $\sigma_{\color{blue}{train}}$ değerlerini “öğrendi”
    * bu öğrenilen değerleri kullanarak hem training set’te hem de test set’te **stateful bir dönüşüm** gerçekleştirdi:
        * $ \large X_{\color{blue}{train-scaled}} =  \frac{X_{\color{blue}{train}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$
        * $ \large X_{\color{red}{test-scaled}} =  \frac{X_{\color{red}{test}} -\mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}$

## (2) 💻 Kendi stateful transformer’ınızı oluşturun

🤔 Peki ya kendi **stateful custom transformer**’ımızı kodlamak istersek?

💪 Kendi class’ımızı yazabiliriz!

# El Yapımı Standardizer

### (2.1) 💻 Custom Standardizer

❓ **Sorular: Kendi class’ınızı kodlama** ❓

1. Kendi `CustomStandardizer` class’ınızı yazın  
    * Scikit Learn’deki `StandardScaler` ile **birebir aynı davranışı** göstermelidir. Yani:
        * $\mu_{\color{blue}{train}}$ ve $\sigma_{\color{blue}{train}}$ değerlerini hesaplayan (“öğrenen”) bir `.fit()` metodu
        * ve bir `.transform()` metodu olmalıdır.

2. Bu class’ı `X_train` üzerinde fit edin

3. Hem `X_train` hem de `X_test` üzerinde transform uygulayın

4. Her şeyi doğru yaptığınızdan emin olmak için `CustomStandardizer`’ınızı Scikit Learn’deki `StandardScaler` ile karşılaştırın!

In [94]:
#########################################
# 1 - CustomStandardizer Class’ını Kodlayın #
#########################################

# TransformerMixin inheritance, fit() ve transform() yöntemlerinden fit_transform() yöntemini oluşturmak için kullanılır
from sklearn.base import TransformerMixin, BaseEstimator


class CustomStandardizer:
    def __init__(self, shrink_factor=1):
        self.shrink_factor = shrink_factor
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y = None):
        """
        X: numpy array veya pandas DataFrame
        """
        X = np.asarray(X)

        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0, ddof=0)  

        return self

    def transform(self, X):
        X = np.asarray(X)

        return (X - self.mean_) / (self.std_ * self.shrink_factor)
        
    def fit_transform(self, X, y = None):
        return self.fit(X,y).transform(X)

    

In [95]:
#########################################
# 2 - CustomStandardizer Class'ını Eğit (Fit) #
#########################################

custom_standardizer = CustomStandardizer()
custom_standardizer.fit(X_train)

In [96]:
#########################################
# 3 - Transform                         #
#########################################
X_train_custom = custom_standardizer.transform(X_train)
X_test_custom = custom_standardizer.transform(X_test)

sk_scaler = StandardScaler()
X_train_sklearn = sk_scaler.fit_transform(X_train)
X_test_sklearn = sk_scaler.transform(X_test)



🧪 **Kodunu Test Et**

In [97]:
from nbresult import ChallengeResult

tmp = CustomStandardizer()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 1 item

test_standardizer.py::TestStandardizer::test_solution PASSED             [100%]

============================== 1 passed in 0.44s ===============================


💯 You can commit your code:

git add tests/standardizer.pickle

git commit -m 'Completed standardizer step'

git push origin master



<details>
<summary>💡 <i>İpuçları</i> (🧪 eğer yukarıdaki testler yalnızca küçük bir farkla başarısız oluyorsa)</summary>

* `np.std()` ile `pd.std()` metotları arasında **küçük bir fark** olduğuna dikkat edin!
    
* Bu [Stack Overflow gönderisi](https://stackoverflow.com/questions/44220290/sklearn-standardscaler-result-different-to-manual-result) yardımcı olabilir 😉
      
</details>

<details>
<summary>💡 <i>Çözüm</i></summary>

Eğer `pd.std` metodunu kullanıyorsanız, scikit-learn’ün `StandardScaler` davranışını birebir taklit edebilmek için `ddof=0` parametresini de kullanmanız gerekir.      
</details>

### (2.2) 💻 Inverse Transform

❓ **Question (Inverse Transform)** ❓

_StandardScaler_ from Scikit Learn has a [`.inverse_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.inverse_transform) method that helps you revert back to the unscaled dataset.

1. Go back to your `CustomStandardizer` class and implement your own `.inverse_transform()` method.

2. Try it on your scaled training set and your scaled test set.

In [98]:
class CustomStandardizer:
    def __init__(self, shrink_factor=1):
        self.shrink_factor = shrink_factor
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):  
        X = np.asarray(X)
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0, ddof=0)
        return self

    def transform(self, X):
        X = np.asarray(X)
        return (X - self.mean_) / (self.std_ * self.shrink_factor)

    def inverse_transform(self, X_scaled):
        X_scaled = np.asarray(X_scaled)
        return X_scaled * (self.std_ * self.shrink_factor) + self.mean_

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [99]:
# Inverse transform’unuzu deneyin ve doğru kodladığınızdan emin olmak için aşağıdaki testi çalıştırın
custom_standardizer = CustomStandardizer()


custom_standardizer.fit(X_train)


X_train_scaled = custom_standardizer.transform(X_train)
X_test_scaled = custom_standardizer.transform(X_test)


X_train_inverse_transformed = custom_standardizer.inverse_transform(X_train_scaled)
X_test_inverse_transformed = custom_standardizer.inverse_transform(X_test_scaled)


🧪 **Kodunu Test Et**

In [100]:
assert np.allclose(X_train_inverse_transformed, X_train)
assert np.allclose(X_test_inverse_transformed, X_test)

### (2.3) 💻 Özel (custom) pipeline tamamlandı!

💪 Scikit-Learn’in `StandardScaler`’ını birebir kopyalamayı başardık.

🌶 Şimdi biraz daha heyecan katalım!

❓ **Soru: Önceki `CustomStandardizer` özel transformer’ını bir shrink factor ile geliştirin** ❓

`CustomStandardizer(shrink_factor = 1)` class’ı, ölçeklemeyi daha güçlü hale getirmek için **ek bir argüman** almalıdır. Bu durumda ölçekleme, $\sigma_{\color{blue}{train}}$ ile **orantılı** olacaktır 👇:

- $ \large X_{\color{blue}{train-scaled}} =  \left(\frac{X_{\color{blue}{train}} - \mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}\right) \times \frac{1}{shrink\ factor}$
- $ \large X_{\color{red}{test-scaled}} =  \left(\frac{X_{\color{red}{test}} - \mu_{\color{blue}{train}}}{\sigma_{\color{blue}{train}}}\right) \times \frac{1}{shrink\ factor}$

In [101]:
###################################
# Custom Standardizer             #
###################################
class CustomStandardizer:
    def __init__(self, shrink_factor=1):
        self.shrink_factor = shrink_factor
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):
        X = np.asarray(X)

        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0, ddof=0)  

        return self

    def transform(self, X):
        X = np.asarray(X)

        return (X - self.mean_) / (self.std_ * self.shrink_factor)

    def inverse_transform(self, X_scaled):
        X_scaled = np.asarray(X_scaled)
        return X_scaled * (self.std_ * self.shrink_factor) + self.mean_

        
    def fit_transform(self, X, y=None):
        return self.fit(X,y).transform(X)


🧪 **Yeni `CustomStandardizer` özel transformer’ınızı test edin** (`shrink_factor = 2` ile):

- `X_train` üzerinde fit edin
- Hem `X_train` hem de `X_test` üzerinde transform uygulayın
- Dönüştürülmüş DataFrame’leri `X_train_transformed` ve `X_test_transformed` değişkenlerinde saklayın

# El Yapımı Standardizer

In [102]:
custom_standardizer = CustomStandardizer(shrink_factor = 2)


custom_standardizer.fit(X_train)

X_train_transformed = custom_standardizer.transform(X_train)
X_test_transformed  = custom_standardizer.transform(X_test)


🧪 **Kodunu Test Et**

In [103]:
from nbresult import ChallengeResult

tmp = CustomStandardizer(shrink_factor=2).fit(X_train)
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'new_standardizer',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 1 item

test_new_standardizer.py::TestNewStandardizer::test_solution PASSED      [100%]

============================== 1 passed in 0.30s ===============================


💯 You can commit your code:

git add tests/new_standardizer.pickle

git commit -m 'Completed new_standardizer step'

git push origin master



In [104]:
# Aşağıdaki hücreleri çalıştırarak doğru dönüşümleri elde ettiğinizden emin olun
truth_train = np.array([
     [-0.612372, -0.612372, -0.612372],
   [ 0.000000, 0.000000, 0.000000],
[0.612372, 0.612372, 0.612372]
 ])
truth_test = np.array([
     [-0.612372, -1.837117, -3.061862],
   [ 0.        , -1.224745, -2.449490],
     [ 0.612372, -0.612372,  1.224745]])

In [105]:
# # Assert’ler
np.allclose(X_train_transformed, truth_train)

True

In [106]:
# # Assert - Test
np.allclose(X_test_transformed, truth_test)

True

❓ **Soru: Önceki `FeatureAverager` özel transformer’ını “tweak” edin** ❓

Bu değiştirilmiş `FeatureAverager()` class’ı:
- hâlâ üç farklı feature’ın ortalamasını hesaplar...
- ...ve ardından sonucu her satır için maksimum değere böler

_Not_: Bu işlemi yorumlamaya çalışmayın. Gerçek bir anlamı yok; amacımız yalnızca özel bir class kodlama becerilerimizi pratik etmek.

$$(X_1, X_2, X_3) \rightarrow \frac{mean(X_1 + X_2 + X_3)}{max(X_1, X_2, X_3)}$$

_Not_: Burada **stateful** mı yoksa **stateless** bir dönüşümle mi uğraştığımızı dikkatlice düşünün (yani, *stateful* `StandardScaler`’da yaptığımız gibi dönüşümden önce tüm training verisi hakkında bilgi öğrenmemiz gerekiyor mu, yoksa yukarıdaki *stateless* `feature_averager` gibi mi çalışıyor?). İlki söz konusuysa `.fit()` için bir şeyler yazmamız gerekir; değilse gerekmez!

In [116]:
###################################
# Feature Averager                #
###################################
class FeatureAverager:
    def __init__(self, feature_indices=(0, 1, 2)):
        self.feature_indices = feature_indices

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = np.asarray(X)

        
        selected = X[:, self.feature_indices]

        
        row_mean = selected.mean(axis=1)

       
        row_max = selected.max(axis=1)

        transformed = row_mean / row_max

        return transformed.reshape(-1, 1)

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)



    

🧪 **Test you `FeatureAverager` custom transformer** by fitting on `X_train`  and transforming both `X_train` and `X_test`

In [117]:
X_train_transformed = feature_averager.fit_transform(X_train)
X_test_transformed = feature_averager.transform(X_test)
#print(X_train_transformed.shape)
#print(X_test_transformed.shape)


In [118]:
from nbresult import ChallengeResult

tmp = FeatureAverager()
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))

result = ChallengeResult(
    'feature_averager',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 1 item

test_feature_averager.py::TestFeatureAverager::test_solution PASSED      [100%]

============================== 1 passed in 0.27s ===============================


💯 You can commit your code:

git add tests/feature_averager.pickle

git commit -m 'Completed feature_averager step'

git push origin master



❓ **Soru (Feature Union)** ❓

1. Hem `CustomStandardizer` hem de `FeatureAverager` kullanarak bir `FeatureUnion` pipeline oluşturun ve bunu `pipeline` adlı bir değişkende saklayın.

2. Pipeline’ı `X_train` üzerinde fit edin ve hem `X_train` hem de `X_test` üzerinde transform uygulayın (`shrink_factor = 3`).

3. Bu challenge’ın son testini geçtiğinizden emin olun.

In [119]:
#####################
# 1 - Feature Union #
#####################
class CustomStandardizer:
    def __init__(self, shrink_factor=1):
        self.shrink_factor = shrink_factor
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):   
        X = np.asarray(X)
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0, ddof=0)
        return self

    def transform(self, X):
        X = np.asarray(X)
        return (X - self.mean_) / (self.std_ * self.shrink_factor)

    def inverse_transform(self, X_scaled):
        X_scaled = np.asarray(X_scaled)
        return X_scaled * (self.std_ * self.shrink_factor) + self.mean_

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)




In [122]:
#########################
# 2 - Fit ve Transform #
#########################
pipeline = Pipeline([
    ("features", FeatureUnion([
        ("standardizer", CustomStandardizer(shrink_factor=3)),
        ("averager", FeatureAverager((0, 1, 2)))
    ]))
])
pipeline.fit(X_train)
X_train_transformed = pipeline.transform(X_train)
X_test_transformed = pipeline.transform(X_test)


/home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/lib/python3.12/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [123]:
from nbresult import ChallengeResult

tmp = pipeline
tmp_train = np.array(tmp.fit_transform(X_train))
tmp_test = np.array(tmp.transform(X_test))



result = ChallengeResult(
    'feature_union_custom_transformers',
    X_train_transformed=tmp_train,
    X_test_transformed=tmp_test
)

result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /home/zeynepdil/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /home/zeynepdil/code/S17D1-S-Data-hand-made-standardizer/tests
plugins: typeguard-4.4.2, anyio-4.8.0, dash-3.3.0
collecting ... collected 1 item

test_feature_union_custom_transformers.py::TestFeatureUnionCustomTransformers::test_solution PASSED [100%]

============================== 1 passed in 0.29s ===============================


💯 You can commit your code:

git add tests/feature_union_custom_transformers.pickle

git commit -m 'Completed feature_union_custom_transformers step'

git push origin master



🏁 Tebrikler! Kendi Transformer’ınızı nasıl oluşturacağınızı keşfettiniz!

💾 Notebook’unuzu `git add / commit / push` yapmayı unutmayın...

🚀 ... ve bir sonraki challenge’a geçin!